In [2]:
from Bio.PDB import *
from utils.carb_utils import *
#import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix as dm


#-alternate_3_letter_codes pdb_sugar
#-out:level 100

#-beta
#-auto_detect_glycan_connections
#-alternate_3_letter_codes pdb_sugar



import os
import numpy as np
import pandas as pd
import copy

BOND_CUT = 1.75
INTERACT = 4.5

input_dir = "./"
os.chdir(input_dir)

In [3]:
ls = os.listdir('./pdb_pre/')
print(len(ls))

1935


In [4]:
def get_coor(structure):
    coor = []
    resid = []
    

    models = structure.get_models()
    models = list(models)
    if len(models) == 0:
        return [],[];
    
    for m in range(len(models)):

        chains = list(models[m].get_chains())
        for c in range(len(chains)):
            residues = list(chains[c].get_residues())
            for r in range(len(residues)):

                res = residues[r].get_resname()
                if res == 'HOH':
                    continue;

                atoms = list(residues[r].get_atoms())

                for a in range(len(atoms)):
                    at = atoms[a]

                    if 'H' in at.get_name():
                        continue;

                    #print(str(residues[r].get_parent().id).strip())

                    coor.append( at.get_coord() )
                    resid.append( [ str(at.get_name()), str(residues[r].id[1]).strip(), str(chains[c].id).strip(), str(residues[r].get_resname()) ] )
                      
        #print(len(coor))
        return np.array(coor), resid

In [5]:
def find_interactChains(coor,res,INTERACT=1.75):
    #determine chain-chain interactions
    d = dm(coor,coor) < INTERACT
    a = np.array( np.where(d == 1) )
    a = np.array(a)

    #chain_int = {}
    chain_int = ''
    for ii in range(a.shape[1]):

        res1 = res[ a[0,ii] ]
        res2 = res[ a[1,ii] ]

        #print(res1,res2)

        #chain_int[res1[1]] = res2[1]
        ope = res1[1] + ',' + res2[1]
        if ope not in chain_int:
            chain_int +=ope + '\n'

        #chain_int.update({res1[1]:res2[2]})

        #bad_c.append( res_c[a[0,ii]] )
        #print(res_c[a[0,ii]], res_p[a[1,ii]])

    #chain_int

    return chain_int

In [6]:
#enter coordinates and chain and get the polymer object instance
def chain_to_poly(my_chain,coor,res):
    """
    params:
        chain (str): chain identifier
        coor (arr n x 3): coordinates
        res (arr n x 4): residue information of each atom (aname, resnum, chain, resname)
    return:
        polymer
    """


    polymer = []
    c_resnum = -1;
    c_resname = ''
    n = []
    c = []
    
    
    for i in range(len(res)):
        ii = res[i]
        aname = ii[0]
        resnum = ii[1]
        chain = ii[2]
        resname = ii[3]
        
        

        if c_resnum != resnum:
            if c_resnum != -1 and len(n) > 1:
                #print(str(c_resnum),c_resname,np.array(c),n)
                m = mono(str(c_resnum),c_resname,np.array(c),n)
                polymer.append(m)
            #reset
            c_resnum = resnum;
            c_resname = resname
            n = []
            c = []

        if chain == my_chain:

            n.append(aname)
            c.append(coor[i])

    
    if len(n) > 1:
        m = mono(str(c_resnum),c_resname,np.array(c),n)
        polymer.append(m)

    return poly(polymer)

In [7]:
fasta = ''

parser=PDBParser()

tor_bois = []


for ii in range(len(ls)):
    pdb = ls[ii]
    
    if ii % 500 == 0:
        print(ii)
    
    if '_carb' not in pdb:
        continue;
        
    p = pdb.split('.')[0][:4]
    
    #if ii == 2:
    #    break;
    #try:
    structure=parser.get_structure("carb", 'pdb_pre/'+ pdb)
    print(ii,pdb)

    coor, res = get_coor(structure)
    if len(coor) == 0:
        print('nope')
        continue
    #inter = find_interactChains(coor,res)
    #print(inter)

    chains = []
    for ii in res:
        #print(ii)
        #break;
        if ii[2] not in chains:
            chains.append(ii[2])
    #print(chains)
    for c in chains:
        try:
            po = chain_to_poly(c,coor,res)
            #print(len(po))
            if len(po) > 1:
                tor_bois.append([pdb,c])

            np.save('npy_pre/' + p + '_' + c + '.npy',po)
        except:
            continue;

        #fasta += get_fasta(structure,p)
    #except:
        #bad.append(pdb)
    #    print('nope')
        
    #if ii >= 10:
    #    break;
    
    
#f = open('pdb_pre/all_fasta.fa','w+')
#f.write(fasta)
#f.close()

0
7 3ZOA_carb.pdb
8 1HJV_carb.pdb
10 1HJW_carb.pdb
11 4J2S_carb.pdb
14 1V6W_carb.pdb
16 3bkl_carb.pdb
18 1V6V_carb.pdb
22 5KJQ_carb.pdb
24 7XDQ_carb.pdb
25 5KK7_carb.pdb
26 1GZT_carb.pdb
30 5afn_carb.pdb
33 4XUT_carb.pdb
35 4Y1V_carb.pdb
38 5GQV_carb.pdb
41 6TCW_carb.pdb
43 2XR5_carb.pdb
45 2XQO_carb.pdb
46 5O2N_carb.pdb
49 1urg_carb.pdb
50 5O2O_carb.pdb
60 3M5H_carb.pdb
64 3M5I_carb.pdb
70 2DT2_carb.pdb
73 2DT3_carb.pdb
75 1nk4_carb.pdb
78 1nk5_carb.pdb
81 2DWI_carb.pdb
82 4ufk_carb.pdb
83 5KKB_carb.pdb
85 6Q27_carb.pdb
89 6BKR_carb.pdb
90 6BJ5_carb.pdb
96 8FRG_carb.pdb
103 5IKQ_carb.pdb
nope
109 1PUU_carb.pdb
112 2ZHN_carb.pdb
115 3L2L_carb.pdb
126 3E7J_carb.pdb
129 6N3R_carb.pdb
137 4M44_carb.pdb
138 5KGP_carb.pdb
141 4ZLT_carb.pdb
146 1ukt_carb.pdb
150 4I4V_carb.pdb
155 1UIH_carb.pdb
158 2QZ2_carb.pdb
162 1JSW_carb.pdb
163 5VKS_carb.pdb
169 2aez_carb.pdb
171 4XLF_carb.pdb
174 1K7T_carb.pdb
176 1K7U_carb.pdb
177 3C7O_carb.pdb
179 7YWF_carb.pdb
182 4U36_carb.pdb
nope
183 4Z6A_carb.pd

In [8]:
f = open('pdb_pre/tor_bois.txt','w+')
f.write('pdb,chain\n')
for ii in tor_bois:
    f.write(ii[0] + ',' + ii[1] + '\n')
f.close()